<a href="https://colab.research.google.com/github/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-02/lista02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Lista 02 - Tratamento de dados
        
> Yanna Torres Gonçalves
>
> Mestrado em Ciências da Computação
>
> Matrícula: 587299

In [ ]:
!pip install duckdb

In [ ]:
import pandas as pd
import numpy as np
import re
import duckdb

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

## a) Ler o dataset fakeTelegram.BR_2022.csv

Utilizamos o mesmo dataset da lista passada, acessando pelo mesmo link.

[https://github.com/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-01/data/fakeTelegram.BR_2022.csv](https://github.com/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-01/data/fakeTelegram.BR_2022.csv)

In [ ]:
data_url = "https://media.githubusercontent.com/media/yanna-torres/CKP9011-ciencia-de-dados/refs/heads/lista-01/data/fakeTelegram.BR_2022.csv"

df = pd.read_csv(data_url)

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
df.info()

## b) Remova os trava-zaps

Utilizamos a mesma função da lista 01 criada para validar os dados da coluna `trava_zap`. Após atualizar a coluna, criamos um novo dataframe com os dados sem trava-zaps.

In [ ]:
def has_trava_zap(message):
  if not isinstance(message, str):
    return False  # Garante que só analisa strings

  # Regra 1: muito longo
  if len(message) > 10000:
    return True

  # Regra 2: caracteres invisíveis
  invisible = ''.join(chr(c) for c in range(0x200B, 0x200F + 1)) + '\uFEFF'
  if sum(message.count(c) for c in invisible) > 100:
    return True

  # Regra 3: emojis repetidos
  emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF"
                              "\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+", flags=re.UNICODE)
  emojis = emoji_pattern.findall(message)
  if any(len(e) > 20 for e in emojis):
    return True

  # Regra 4: baixa diversidade
  unique = set(message)
  if len(unique) < 10 and len(message) > 500:
    return True

  return False

In [ ]:
df['trava_zap'] = df['text_content_anonymous'].apply(has_trava_zap)

In [ ]:
df_clean = df[df['trava_zap'] == False]

In [ ]:
df_clean.drop(columns=['trava_zap'], inplace=True)

In [ ]:
df_clean.head(10)

In [ ]:
df_clean.describe()

In [ ]:
df_clean.info()

---

Adicionando uma coluna do texto sem stop words para facilitar itens futuros.

In [ ]:
stop_words = set(stopwords.words('portuguese'))

def clean_text(text):
  if not isinstance(text, str):
    return ''
  text_no_punct = text.translate(str.maketrans('', '', string.punctuation))
  tokens = word_tokenize(text_no_punct)
  filtered = [word for word in tokens if word.lower() not in stop_words]
  return ' '.join(filtered)

df_clean['text_no_stopwords'] = df_clean['text_content_anonymous'].apply(clean_text)

---

## c) Exportar os dados para um arquivo Parquet

Para exportar os dados para um arquivo Parquet, utilizamos a função `to_parquet` do Pandas.

In [ ]:
df_clean.to_parquet("fakeTelegramBR_2022_clean.parquet")

## d) Exportar os dados para o DuckDB

Para exportar os dados para um banco do DuckDB, utilizamos o comando SQL:

```sql
CREATE TABLE nome_da_table AS SELECT * FROM nome_do_dataframe
```

[Documentação DuckDB](https://duckdb.org/docs/stable/guides/python/import_pandas.html)

In [ ]:
conn = duckdb.connect()

conn.execute("CREATE TABLE messages AS SELECT * FROM df_clean")

In [ ]:
result = conn.execute("SELECT * FROM messages LIMIT 10").df()

In [ ]:
result

## e) Utlizando o DuckDB recupere

### 1. A quantidade de mensagens

Consultamos o total de mensagens registradas na base de dados utilizando uma consulta SQL simples com `COUNT(*)`.

In [ ]:
quant_messages = conn.execute("SELECT COUNT(*) AS total_messages FROM messages").df()
quant_messages

### 2. A quantidade de usuários

Utilizamos a função `COUNT(DISTINCT ...) `para contar o número de usuários únicos na base, identificados pelo campo id_member_anonymous.

In [ ]:
quant_users = conn.execute("SELECT COUNT(DISTINCT id_member_anonymous) AS total_users FROM messages").df()
quant_users

### 3. A quantidade de grupos

Aplicamos `COUNT(DISTINCT id_group_anonymous)` para identificar quantos grupos distintos estão presentes na base de mensagens.

In [ ]:
quant_groups = conn.execute("SELECT COUNT(DISTINCT id_group_anonymous) AS total_groups FROM messages").df()
quant_groups

### 4. Quantidade de mensagens que possuem apenas texto

Selecionamos as mensagens que possuem conteúdo textual (`text_content_anonymous IS NOT NULL`) e não possuem mídia `(has_media = False`) para contar quantas são exclusivamente de texto.

In [ ]:
quant_messages_only_text = conn.execute("""
SELECT COUNT(*) AS total_messages
FROM messages
WHERE text_content_anonymous IS NOT NULL AND has_media = False
""").df()
quant_messages_only_text

### 5. Quantidade de mensagens contendo mídias

Filtramos as mensagens que possuem algum tipo de mídia (`has_media = True`) para obter a quantidade total desse tipo de conteúdo na base.

In [ ]:
quant_messages_with_media = conn.execute("""
SELECT COUNT(*) AS total_messages
FROM messages
WHERE has_media = True
""").df()
quant_messages_with_media

### 6. Quantidade de mensagens por tipo de mídia (jpg, mp4 etc)

Agrupamos as mensagens com mídia de acordo com o tipo (`media_type`) e utilizamos `COUNT(*)` para contabilizar quantas mensagens há de cada tipo (como imagens JPG, vídeos MP4, entre outros).

In [ ]:
quant_messages_per_media_type_df = conn.execute("""
SELECT media_type, COUNT(*) AS total_messages
FROM messages
WHERE has_media = True
GROUP BY media_type
""").df()
quant_messages_per_media_type_df

In [ ]:
quant_messages_per_media_type_df.sort_values(by='total_messages', ascending=False).head(10)

### 7. Quantidade de mensagens por estado

### 8. Quantidade de usuários por estado

### 9. Relação quantidade de usuários por quantidade de mensagens por estado

### 10. Quantidade de mensagens por país

### 11. Quantidade de mensagens Brasil X Países Estrangeiros

O dataset não possui informações de telefone, de identificação do estado ou de país de origem de cada mensagem.

In [ ]:
columns = conn.execute("SELECT * FROM messages LIMIT 0").df().columns
columns

### 12. As 30 URLs que mais se repetem (mais compartilhadas)

Selecionamos as URLs não nulas e agrupamos pelo valor da URL para contar quantas vezes cada uma foi compartilhada. Em seguida, ordenamos em ordem decrescente e limitamos o resultado às 30 mais frequentes.

In [ ]:
top_urls = conn.execute("""
SELECT media_url as url, COUNT(*) AS total
FROM messages
WHERE url IS NOT NULL
GROUP BY url
ORDER BY total DESC
LIMIT 30
""").df()
top_urls

### 13. Os 30 domínios que mais se repetem (mais compartilhados)

Utilizamos expressões regulares para extrair o domínio das URLs compartilhadas, desconsiderando o protocolo (`http://` ou `https://`) e o prefixo `www.`. Em seguida, agrupamos por domínio, contamos as ocorrências e listamos os 30 mais frequentes.

In [ ]:
regex_domains = "LOWER(regexp_extract(regexp_replace(media_url, 'https?://(www\.)?', ''), '^([^/]+)', 1))"
top_domains = conn.execute(f"""
SELECT
    {regex_domains} AS domain,
    COUNT(*) AS total
FROM messages
WHERE media_url IS NOT NULL
GROUP BY domain
ORDER BY total DESC
LIMIT 30
""").df()
top_domains

### 14. Os 30 usuários mais ativos

Agrupamos as mensagens pelo `id_member_anonymous` e contamos o número de mensagens enviadas por cada usuário. O resultado foi ordenado em ordem decrescente, mostrando os 30 usuários com maior número de mensagens.

In [ ]:
top_users = conn.execute("""
SELECT id_member_anonymous, COUNT(*) AS total
FROM messages
WHERE id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
ORDER BY total DESC
LIMIT 30
""").df()
top_users

### 15. Os 30 usuários que mais compartilharam texto

Selecionamos os usuários que compartilharam mensagens de texto não nulas e não vazias. Contamos quantas mensagens de texto cada usuário enviou e listamos os 30 mais ativos nesse tipo de conteúdo.

In [ ]:
top_text_users = conn.execute("""
SELECT id_member_anonymous, COUNT(*) AS total
FROM messages
WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous != '' AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
ORDER BY total DESC
LIMIT 30
""").df()
top_text_users

### 16. Os 30 usuários que mais compartilharam mídias

Selecionamos os usuários que compartilharam mensagens com mídias, verificando tanto o campo `has_media` quanto a presença de URLs de mídia não nulas e não vazias. Contamos o número de mensagens de mídia enviadas por cada usuário e listamos os 30 mais ativos nesse tipo de conteúdo.

In [ ]:
top_media_users = conn.execute("""
SELECT id_member_anonymous, COUNT(*) AS total
FROM messages
WHERE (has_media = TRUE OR (media_url IS NOT NULL AND media_url != ''))
AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
ORDER BY total DESC
LIMIT 30
""").df()
top_media_users

### 17. As 30 mensagens mais compartilhadas

Selecionamos as mensagens que foram compartilhadas mais de uma vez, contando o número de ocorrências de cada mensagem. O resultado foi ordenado para exibir as 30 mensagens com o maior número de compartilhamentos.

In [ ]:
top_shared_messages = conn.execute("""
SELECT text_content_anonymous as message, COUNT(*) AS total_shares
FROM messages WHERE message IS NOT NULL OR message != ''
GROUP BY message
ORDER BY total_shares DESC
LIMIT 30
""").df()
top_shared_messages

### 18. As 30 mensagens mais compartilhadas em grupos diferentes

Contabilizamos as mensagens que foram compartilhadas em diferentes grupos, utilizando `COUNT(DISTINCT id_group_anonymous)` para garantir que cada grupo fosse contado apenas uma vez. O resultado exibe as 30 mensagens com maior alcance em termos de grupos distintos.

In [ ]:
top_shared_messages_in_groups = conn.execute("""
SELECT text_content_anonymous as message, COUNT(DISTINCT id_group_anonymous) AS total_groups
FROM messages
WHERE message IS NOT NULL AND id_group_anonymous IS NOT NULL AND message != ''
GROUP BY message
ORDER BY total_groups DESC
LIMIT 30
""").df()
top_shared_messages_in_groups

### 19. Mensagens idênticas compartilhadas pelo mesmo usuário (e suas quantidades)

Identificamos mensagens de texto idênticas enviadas pelo mesmo usuário, contando quantas vezes cada mensagem foi repetida. O resultado exibe apenas as mensagens repetidas mais de uma vez, ordenadas pela quantidade de ocorrências.

In [ ]:
repeated_messages_by_user = conn.execute("""
SELECT
    id_member_anonymous,
    text_content_anonymous,
    COUNT(*) AS total
FROM messages
WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous != ''
      AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous, text_content_anonymous
HAVING COUNT(*) > 1
ORDER BY total DESC
""").df()
repeated_messages_by_user

### 20. Mensagens idênticas compartilhadas pelo mesmo usuário em grupos distintos (e suas quantidades)

Contabilizamos as mensagens de texto idênticas enviadas pelo mesmo usuário em diferentes grupos. Utilizamos `COUNT(DISTINCT id_group_anonymous)` para contar os grupos distintos onde cada mensagem foi compartilhada mais de uma vez. O resultado exibe as mensagens que foram repetidas em múltiplos grupos, ordenadas pela quantidade de grupos distintos.

In [ ]:
repeated_messages_by_user_groups = conn.execute("""
SELECT
    id_member_anonymous,
    text_content_anonymous as message,
    COUNT(DISTINCT id_group_anonymous) AS total_groups
FROM messages
WHERE text_content_anonymous IS NOT NULL
      AND message != ''
      AND id_member_anonymous IS NOT NULL
      AND id_group_anonymous IS NOT NULL
GROUP BY id_member_anonymous, message
HAVING COUNT(DISTINCT id_group_anonymous) > 1
ORDER BY total_groups DESC
""").df()
repeated_messages_by_user_groups

### 21. Os 30 unigramas, bigramas e trigramas mais compartilhados

Para identificar os 30 unigramas, bigramas e trigramas mais compartilhados, extraímos os tokens (palavras) das mensagens de texto e aplicamos contagens em três etapas.

**Unigramas (1-gram):** Analisamos as palavras isoladas nas mensagens, excluindo palavras vazias e stopwords, para contar as mais frequentes.

In [ ]:
top_30_1gram = conn.execute("""
SELECT
    word AS ngram,
    COUNT(*) AS total
FROM (
    SELECT UNNEST(STRING_SPLIT(text_no_stopwords, ' ')) AS word
    FROM messages
    WHERE text_no_stopwords IS NOT NULL AND text_no_stopwords != ''
)
GROUP BY word
ORDER BY total DESC
LIMIT 30;
""").df()
top_30_1gram

**Bigramas (2-gram):** Formamos pares consecutivos de palavras (bigramas) e contamos sua ocorrência nas mensagens.

In [ ]:
query = """
WITH tokens AS (
    SELECT word
    FROM (
        SELECT unnest(string_split_regex(lower(text_content_anonymous), '[^\\p{L}0-9]+')) as word
        FROM messages
        WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous != ''
    )
    WHERE word NOT LIKE 'http%' AND word NOT LIKE 'www%' AND word NOT LIKE 'com%' AND word NOT LIKE 't.me%' AND word NOT LIKE 'youtu%' AND length(word) > 1
),
ngrams AS (
    SELECT
        word || ' ' || lead(word, 1) OVER () as two_gram
    FROM tokens
)
SELECT
    two_gram as "2gram",
    count(*) as count
FROM ngrams
WHERE two_gram NOT LIKE '% % %'
GROUP BY two_gram
ORDER BY count DESC
LIMIT 30;
"""
top_2grams = conn.execute(query).df()
top_2grams

**Trigramas (3-gram):** Formamos tripletos consecutivos de palavras (trigramas) e contabilizamos os mais repetidos.

In [ ]:
query = """
WITH tokens AS (
    SELECT word
    FROM (
        SELECT unnest(string_split_regex(lower(text_content_anonymous), '[^\\p{L}0-9]+')) AS word
        FROM messages
        WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous != ''
    )
    WHERE word NOT LIKE 'http%'
      AND word NOT LIKE 'www%'
      AND word NOT LIKE 'com%'
      AND word NOT LIKE 't.me%'
      AND word NOT LIKE 'youtu%'
      AND length(word) > 1
),
ngrams AS (
    SELECT
        word || ' ' || lead(word, 1) OVER () || ' ' || lead(word, 2) OVER () AS three_gram
    FROM tokens
)
SELECT
    three_gram AS "3gram",
    count(*) AS count
FROM ngrams
WHERE three_gram NOT LIKE '% % % %'
GROUP BY three_gram
ORDER BY count DESC
LIMIT 30;
"""
top_3grams = conn.execute(query).df()
top_3grams

### 22. As 30 mensagens mais positivas (distintas)

Selecionamos as mensagens com uma pontuação de sentimento (`score_sentiment`) superior ou igual a 0.05, indicando que são mensagens com tonalidade positiva. Excluímos duplicatas e ordenamos as mensagens pela pontuação de sentimento em ordem decrescente, exibindo as 30 mensagens mais positivas.

In [ ]:
top_positive_messages = conn.execute("""
SELECT DISTINCT text_content_anonymous, score_sentiment
FROM messages
WHERE text_content_anonymous IS NOT NULL
  AND text_content_anonymous != ''
  AND score_sentiment IS NOT NULL AND score_sentiment >= 0.05
ORDER BY score_sentiment DESC
LIMIT 30;
""").df()
top_positive_messages

### 23. As 30 mensagens mais negativas (distintas)

Selecionamos as mensagens com uma pontuação de sentimento (`score_sentiment`) inferior ou igual a 0.05, indicando que são mensagens com tonalidade negativa. Excluímos duplicatas e ordenamos as mensagens pela pontuação de sentimento em ordem crescente, exibindo as 30 mensagens mais negativas.

In [ ]:
top_negative_messages = conn.execute("""
SELECT DISTINCT text_content_anonymous, score_sentiment
FROM messages
WHERE text_content_anonymous IS NOT NULL
  AND text_content_anonymous != ''
  AND score_sentiment IS NOT NULL AND score_sentiment <= 0.05
ORDER BY score_sentiment ASC
LIMIT 30;
""").df()
top_negative_messages

### 24. O usuário mais otimista

Identificamos o usuário mais otimista, calculando a média da pontuação de sentimento (`score_sentiment`) para cada usuário.

In [ ]:
top_positive_user = conn.execute("""
SELECT
    id_member_anonymous,
    AVG(score_sentiment) AS avg_sentiment,
    COUNT(*) AS total_messages
FROM messages
WHERE score_sentiment IS NOT NULL
  AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
ORDER BY avg_sentiment DESC
LIMIT 1;
""").df()
top_positive_user

In [ ]:
top_positive_user_10 = conn.execute("""
SELECT
    id_member_anonymous,
    AVG(score_sentiment) AS avg_sentiment,
    COUNT(*) AS total_messages
FROM messages
WHERE score_sentiment IS NOT NULL
  AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
HAVING COUNT(*) >= 10
ORDER BY avg_sentiment DESC
LIMIT 1;
""").df()
top_positive_user_10

### 25. O usuário mais pessimista

In [ ]:
top_negative_user = conn.execute("""
SELECT
    id_member_anonymous,
    AVG(score_sentiment) AS avg_sentiment,
    COUNT(*) AS total_messages
FROM messages
WHERE score_sentiment IS NOT NULL
  AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
ORDER BY avg_sentiment ASC
LIMIT 1;
""").df()
top_negative_user

In [ ]:
top_negative_user_10 = conn.execute("""
SELECT
    id_member_anonymous,
    AVG(score_sentiment) AS avg_sentiment,
    COUNT(*) AS total_messages
FROM messages
WHERE score_sentiment IS NOT NULL
  AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
HAVING COUNT(*) >= 10
ORDER BY avg_sentiment ASC
LIMIT 1;
""").df()
top_negative_user_10

### 26. As 30 maiores mensagens

Selecionamos as 30 mensagens com o maior número de caracteres. Excluímos mensagens vazias ou nulas e ordenamos as mensagens pela quantidade de caracteres em ordem decrescente, para destacar as mensagens mais longas.

In [ ]:
top_longest_messages = conn.execute("""
SELECT
    id_message,
    id_member_anonymous,
    LENGTH(text_content_anonymous) AS message_length,
    text_content_anonymous
FROM messages
WHERE text_content_anonymous IS NOT NULL
  AND text_content_anonymous != ''
ORDER BY message_length DESC
LIMIT 30;
""").df()
top_longest_messages

### 27. As 30 menores mensagens

Selecionamos as 30 mensagens com o menor número de caracteres. Excluímos mensagens vazias ou nulas e ordenamos as mensagens pela quantidade de caracteres em ordem crescente, para destacar as mensagens mais curtas.

In [ ]:
top_shortest_messages = conn.execute("""
SELECT
    id_message,
    id_member_anonymous,
    LENGTH(text_content_anonymous) AS message_length,
    text_content_anonymous
FROM messages
WHERE text_content_anonymous IS NOT NULL
  AND text_content_anonymous != ''
ORDER BY message_length ASC
LIMIT 30;
""").df()
top_shortest_messages

### 28. O dia em que foi publicado a maior quantidade de mensagens

Identificamos o dia em que foi registrada a maior quantidade de mensagens. Para isso, agrupamos as mensagens pela data de publicação e selecionamos o dia com o maior número de mensagens, com base na contagem de registros.

In [ ]:
day_with_most_messages = conn.execute("""
SELECT
    CAST(date_message AS DATE) AS message_day,
    COUNT(*) AS total_messages
FROM messages
WHERE date_message IS NOT NULL
GROUP BY message_day
ORDER BY total_messages DESC
LIMIT 1;
""").df()
day_with_most_messages

### 29. As mensagens que possuem as palavras “FACÇÃO” e “CRIMINOSA”

Selecionamos as mensagens que contêm as palavras “facção” e “criminosa” no conteúdo de texto. A pesquisa foi realizada de forma a garantir que ambas as palavras estivessem presentes, independentemente de sua capitalização.

In [ ]:
messages_with_faction_and_criminal = conn.execute("""
SELECT
    id_message,
    id_member_anonymous,
    text_content_anonymous
FROM messages
WHERE text_content_anonymous IS NOT NULL
  AND LOWER(text_content_anonymous) LIKE '%facção%'
  AND LOWER(text_content_anonymous) LIKE '%criminosa%'
""").df()
messages_with_faction_and_criminal

### 30. As mensagens que possuem a palavra “SEGURANÇA”

Selecionamos as mensagens que contêm a palavra "segurança" no conteúdo de texto. A pesquisa foi realizada de forma insensível a maiúsculas e minúsculas, garantindo que todas as ocorrências fossem capturadas.

In [ ]:
messages_with_seguranca = conn.execute("""
SELECT
    id_message,
    id_member_anonymous,
    text_content_anonymous
FROM messages
WHERE text_content_anonymous IS NOT NULL
  AND LOWER(text_content_anonymous) LIKE '%segurança%'
""").df()
messages_with_seguranca

---

![MDCC](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQpPMiWU_DtYHDxAG5A3icfDwguLHmjzSqK7w&s)